# 📌 课程作业：HW1（总分 100 分）

## **提交说明**
📩 **提交方式**：请将作业发送至邮箱 **jiqing@cup.edu.cn**

📌 **邮件主题格式**：
```ECO-HW1-姓名-学号```

📂 **文件命名格式**：
```ECO_HW1_姓名_学号.ipynb```

📅 **截止提交日期**：
👉 **2025 年 3 月 24 日（星期一上课前）**

---

## **求解模型**
我们研究的模型如下：
$
\ln(wage) = \beta_0 + \beta_1 \cdot educ + \beta_2 \cdot exper + \beta_3 \cdot expersq + \epsilon
$

- 其中，$educ$ 为 **内生变量**，其 **工具变量** 选择为：
  - $constant$（常数项）
  - $exper$（工作经验）
  - $expersq$（经验的平方）
  - $motheduc$（母亲受教育程度）
  - $fatheduc$（父亲受教育程度）



## **数据导入与预处理**
本次作业使用 **Wooldridge 经济学数据集**，请确保已安装 `wooldridge` 库，并使用以下代码导入 **Mroz** 数据集：

---

In [ ]:
import wooldridge as woo
mroz = woo.dataWoo('mroz') # 加载 Mroz 数据集
mroz = mroz.dropna(subset=['lwage']) # 删除缺失工资数据的观测值
# mroz.head() # 显示前几行数据

·### **任务一：计算两阶段最小二乘的系数方差和 T 值（共 50 分）**

---

### **两阶段最小二乘法（2SLS）**
**直接估计公式**：
$
\beta = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z'y
$

**估计结果**：
$
\hat{\ln(wage)} = 0.0481 + 0.0614 \cdot educ + 0.0442 \cdot exper - 0.0009 \cdot expersq
$

---

#### **任务 1.1：计算系数估计值**
- 估计 **$\beta$ 系数**，即使用 2SLS 方法求解 **回归系数估计值**（10 分）

#### **任务 1.2：假设误差项服从独立同方差**
- 计算 **系数的方差** 和 **T 值**（10 分）

#### **任务 1.3：假设误差项服从独立异方差**
- 计算 **系数的方差** 和 **T 值**（10 分）

#### **任务 1.4：假设误差项存在组内相关性（聚类到 city-level）**
- 计算 **系数的方差（组内异方差聚类）** 和 **T 值**（10 分）

#### **任务 1.5：使用 `statsmodels.api` 对上述三种情况进行检验**
- 采用 `import statsmodels.api as sm` 进行检验，并比较结果（10 分）

---

两阶段直接求解方法：$ \beta = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z'y$

估计值方差：$Var[\beta] = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z' \Omega Z(Z'Z)^{-1}Z'X(X'Z(Z'Z)^{-1}Z'X)^{-1}$, 其中 $\Omega$ 为同方差

In [2]:
import wooldridge as woo
import numpy as np

mroz = woo.dataWoo('mroz') # 加载 Mroz 数据集
mroz = mroz.dropna(subset=['lwage']) # 删除缺失工资数据的观测值
# mroz.head() # 显示前几行数据

In [3]:
## 获取目标变量
ln_wage = np.log(mroz['wage'])
cons = np.ones_like(ln_wage)
educ = mroz['educ']
exper = mroz['exper']
expersq = mroz['expersq']
motheduc = mroz['motheduc']
fatheduc = mroz['fatheduc']

X = np.c_[cons, educ, exper, expersq]
Z = np.c_[cons, exper, expersq, motheduc, fatheduc]
y = ln_wage

2sls estimator（普通标准误） 计算公式：
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta] = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z' \Omega Z(Z'Z)^{-1}Z'X(X'Z(Z'Z)^{-1}Z'X)^{-1}$$

   - 展开式：
$$Var[\beta] = \sigma^2 (X'Z(Z'Z)^{-1}Z'X)^{-1}$$

In [4]:
# 同方差
temp1 = np.linalg.inv(Z.T @ Z)
beta = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X) @ X.T @ Z @ temp1 @ Z.T @ y
u = y - X @ beta
sigma_squared = u.T @ u / (len(y))
# sigma_squared = u.T @ u / (len(y) - X.shape[1])
temp2 = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X)
beta_var = sigma_squared * temp2
beta_var_diag = np.diag(beta_var)

# 计算标准误差，并输出
beta_std = np.sqrt(beta_var_diag)
print(f'cons: {beta[0]:.4f}  {beta_std[0]:.4f}')
print(f'educ: {beta[1]:.4f}  {beta_std[1]:.4f}')
print(f'exper: {beta[2]:.4f}  {beta_std[2]:.4f}')
print(f'expersq: {beta[3]:.4f}  {beta_std[3]:.4f}')

cons: 0.0481  0.3985
educ: 0.0614  0.0313
exper: 0.0442  0.0134
expersq: -0.0009  0.0004


In [5]:
from linearmodels.iv import IV2SLS

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IV2SLS(ln_wage, exog, endog, instruments)
results1 = model.fit(cov_type = "homoskedastic")
print(results1.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    24.653
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0000
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:         homoskedastic                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0481     0.3985     0.1207     0.9039     -0.7329      0.8291
exog.1         0.0442     0.0134     3.3038     0.00

2sls estimator（稳健标准误） 计算公式：
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta] = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z' \Omega Z(Z'Z)^{-1}Z'X(X'Z(Z'Z)^{-1}Z'X)^{-1}$$

   - 展开式：
$$
Var[\hat{\beta}] = (X^{\prime}Z (Z^{\prime}Z)^{-1} Z^{\prime}X)^{-1}
\left[ X^{\prime}Z (Z^{\prime}Z)^{-1}
\left( \sum_{i=1}^{N} Z_i^{\prime} \widehat{u}_i \widehat{u}_i^{\prime} Z_i \right)
(Z^{\prime}Z)^{-1} Z^{\prime}X \right]
(X^{\prime}Z (Z^{\prime}Z)^{-1} Z^{\prime}X)^{-1}
$$

In [6]:
beta = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X) @ X.T @ Z @ temp1 @ Z.T @ y
u = y - X @ beta # 计算残差的平方
sigma_matrix = np.diag(u ** 2)  # 构建对角权重矩阵

temp1 = np.linalg.inv(Z.T @ Z)
temp2 = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X) @ X.T @ Z  @ temp1 @ Z.T
beta_var = temp2 @ sigma_matrix @ temp2.T
beta_var_diag = np.diag(beta_var)

# 计算标准误差，并输出
beta_std = np.sqrt(beta_var_diag)
print(f'cons: {beta[0]:.4f}  {beta_std[0]:.4f}')
print(f'educ: {beta[1]:.4f}  {beta_std[1]:.4f}')
print(f'exper: {beta[2]:.4f}  {beta_std[2]:.4f}')
print(f'expersq: {beta[3]:.4f}  {beta_std[3]:.4f}')

cons: 0.0481  0.4278
educ: 0.0614  0.0332
exper: 0.0442  0.0155
expersq: -0.0009  0.0004


In [7]:
from linearmodels.iv import IV2SLS

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IV2SLS(ln_wage, exog, endog, instruments)
results1 = model.fit(cov_type = "heteroskedastic")  # 使用稳健标准误

print(results1.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    18.611
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0003
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:       heteroskedastic                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0481     0.4278     0.1124     0.9105     -0.7903      0.8865
exog.1         0.0442     0.0155     2.8546     0.00

2sls estimator（聚类标准误） 计算公式：
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta] = (X'Z(Z'Z)^{-1}Z'X)^{-1} X'Z(Z'Z)^{-1}Z' \Omega Z(Z'Z)^{-1}Z'X(X'Z(Z'Z)^{-1}Z'X)^{-1}$$

   - 聚类展开式：
$$
Var[\hat{\beta}] = (X^{\prime}Z (Z^{\prime}Z)^{-1} Z^{\prime}X)^{-1}
\left[ X^{\prime}Z (Z^{\prime}Z)^{-1}
\left( \sum_{g=1}^{G} Z_g^{\prime} \widehat{u}_g \widehat{u}_g^{\prime} Z_g \right)
(Z^{\prime}Z)^{-1} Z^{\prime}X \right]
(X^{\prime}Z (Z^{\prime}Z)^{-1} Z^{\prime}X)^{-1}
$$

In [8]:
# 聚类稳健标准误（残差项不相关，但存在异方差）
N = X.shape[0]  # 总样本数
K = X.shape[1]  # 变量个数

cluster_ids = mroz['city'] # 我们按 'city' 进行聚类
cluster_labels = np.unique(cluster_ids)  # 获取唯一 cluster
G = len(cluster_labels)  # 计算 cluster 数量
S = np.zeros((Z.shape[1], Z.shape[1]))  # 形状为 (K, K) 的零矩阵

# 遍历每个 cluster，计算贡献
for g in cluster_labels:
    idx = (cluster_ids == g)  # 选出属于 cluster g 的观测
    X_g = X[idx, :]  # 选取该 cluster 的 X 矩阵
    Z_g = Z[idx, :]  # 选取该 cluster 的 X 矩阵
    u_g = u[idx].to_numpy().reshape(-1, 1)  # 其中 -1 表示自动计算行数，1 表示只有一列，即转换为列向量。
    S += Z_g.T @ u_g @ u_g.T @ Z_g

temp1 = np.linalg.inv(Z.T @ Z)
temp2 = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X) @ X.T @ Z  @ temp1

# 计算聚类稳健标准误的方差估计
# dof_correction = (G / (G - 1)) * ((N - 1) / (N - K))  # 附加自由度修正
# beta_var_cluster = temp2 @ S @ temp2.T * dof_correction # 进行自由度修正
beta_var_cluster = temp2 @ S @ temp2.T  # 不进行自由度修正
beta_var_diag = np.diag(beta_var_cluster)  # 取主对角线方差
beta_std = np.sqrt(beta_var_diag)  # 计算标准误差

print(f'cons: {beta[0]:.4f}  {beta_std[0]:.4f}')
print(f'educ: {beta[1]:.4f}  {beta_std[1]:.4f}')
print(f'exper: {beta[2]:.4f}  {beta_std[2]:.4f}')
print(f'expersq: {beta[3]:.4f}  {beta_std[3]:.4f}')


cons: 0.0481  0.1236
educ: 0.0614  0.0126
exper: 0.0442  0.0079
expersq: -0.0009  0.0002


In [9]:
from linearmodels.iv import IV2SLS

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IV2SLS(ln_wage, exog, endog, instruments)
iv_model = model.fit(cov_type='clustered', clusters=mroz['city'])
print(iv_model.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                 9.313e+13
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0000
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:             clustered                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0481     0.1236     0.3890     0.6973     -0.1942      0.2904
exog.1         0.0442     0.0079     5.6155     0.00

### **任务二：计算广义矩估计（GMM）的系数方差和 T 值（共 50 分）**

### **广义矩估计（GMM）**
**两步法估计公式**：$\hat{\beta}_{gmm} = (X' Z \Phi Z' X)^{-1} X' Z \Phi Z' y$
- 其中，**最优权重矩阵** 分两步计算：
  - 第一步：$\hat{\epsilon}^{(1)}$ 在 $\Phi = (Z'Z)^{-1}$ 的假设下估计得到
  - 第二步：$\Phi = (Z'\hat{\epsilon}^{(1)} \hat{\epsilon}^{(1)'}Z)^{-1}$

**估计结果**：
$\hat{\ln(wage)} = 0.0477 + 0.0614 \cdot educ + 0.0451 \cdot exper - 0.0009 \cdot expersq$

---

#### **任务 2.1：计算系数估计值**
- 估计 **$\beta$ 系数**，即使用 GMM 方法求解 **回归系数估计值**（10 分）

#### **任务 2.2：假设误差项服从独立同方差**
- 计算 **系数的方差** 和 **T 值**（10 分）

#### **任务 2.3：假设误差项服从独立异方差**
- 计算 **系数的方差** 和 **T 值**（10 分）

#### **任务 2.4：假设误差项存在组内相关性（聚类到 city-level）**
- 计算 **系数的方差（组内异方差聚类）** 和 **T 值**（10 分）

#### **任务 2.5：使用 `statsmodels.api` 对上述三种情况进行检验**
- 采用 `import statsmodels.api as sm` 进行检验，并比较结果（10 分）

---

GMM estimator（普通标准误） 计算公式：$\hat{\beta}^{(2)} = (X' Z \Phi^{(2)} Z' X)^{-1} X' Z \Phi^{(2)} Z' y$
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta^{(2)}] = (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}Z' \Omega Z\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

   - 展开式：
$$Var[\beta] = \sigma^2 (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}Z' Z\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

In [10]:
X = np.c_[cons, mroz['educ'], exper, expersq]
Z = np.c_[cons, mroz['exper'], mroz['expersq'], mroz['motheduc'], mroz['fatheduc']]
Phi_1 = np.linalg.inv(Z.T @ Z)
beta_2sls = np.linalg.inv(X.T @ Z @ Phi_1 @ Z.T @ X) @ X.T @ Z @ Phi_1 @ Z.T @ ln_wage

u_1st = ln_wage - X @ beta_2sls
u_1st = np.array(u_1st)
u_1st_reshaped = u_1st[:, np.newaxis]  # 添加一个新的轴，使其成为 (428, 1)
Z_new= Z*u_1st_reshaped
Phi_2 = np.linalg.inv(Z_new.T @ Z_new / len(ln_wage) )
beta_gmm = np.linalg.inv(X.T @ Z @ Phi_2 @ Z.T @ X) @ X.T @ Z @ Phi_2 @ Z.T @ ln_wage

u = y - X @ beta_gmm
sigma_squared = u.T @ u / (len(y))
# sigma_squared = u.T @ u / (len(y) - X.shape[1])

temp1 = np.linalg.inv(X.T @ Z @ Phi_2 @ Z.T @ X) @ X.T @ Z @ Phi_2 @ Z.T
beta_var = sigma_squared * temp1 @ temp1.T
beta_var_diag = np.diag(beta_var)

# 计算标准误差，并输出
beta_gmm_std = np.sqrt(beta_var_diag)
print(f'cons: {beta_gmm[0]:.4f}  {beta_gmm_std[0]:.4f}')
print(f'educ: {beta_gmm[1]:.4f}  {beta_gmm_std[1]:.4f}')
print(f'exper: {beta_gmm[2]:.4f}  {beta_gmm_std[2]:.4f}')
print(f'expersq: {beta_gmm[3]:.4f}  {beta_gmm_std[3]:.4f}')


cons: 0.0477  0.3985
educ: 0.0611  0.0313
exper: 0.0451  0.0135
expersq: -0.0009  0.0004


In [11]:
from linearmodels.iv import IVGMM

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IVGMM(ln_wage, exog, endog, instruments)
results1 = model.fit(cov_type = "homoskedastic")
print(results1.summary)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   wage   R-squared:                      0.1354
Estimator:                     IV-GMM   Adj. R-squared:                 0.1293
No. Observations:                 428   F-statistic:                    24.855
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0000
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:         homoskedastic                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0477     0.3985     0.1196     0.9048     -0.7334      0.8288
exog.1         0.0451     0.0135     3.3523     0.00

GMM estimator（稳健标准误） 计算公式：$\hat{\beta}^{(2)} = (X' Z \Phi^{(2)} Z' X)^{-1} X' Z \Phi^{(2)} Z' y$
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta^{(2)}] = (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}Z' \Omega Z\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

   - 展开式：
$$Var[\beta] = (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}\left( \sum_{i=1}^{N} Z_i^{\prime} \widehat{u}_i \widehat{u}_i^{\prime} Z_i \right)\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

In [12]:
X = np.c_[cons, mroz['educ'], exper, expersq]
Z = np.c_[cons, mroz['exper'], mroz['expersq'], mroz['motheduc'], mroz['fatheduc']]
Phi_1 = np.linalg.inv(Z.T @ Z)
beta_2sls = np.linalg.inv(X.T @ Z @ Phi_1 @ Z.T @ X) @ X.T @ Z @ Phi_1 @ Z.T @ ln_wage

u_1st = ln_wage - X @ beta_2sls
u_1st = np.array(u_1st)
u_1st_reshaped = u_1st[:, np.newaxis]  # 添加一个新的轴，使其成为 (428, 1)
Z_new= Z*u_1st_reshaped
Phi_2 = np.linalg.inv(Z_new.T @ Z_new / len(ln_wage) )
beta_gmm = np.linalg.inv(X.T @ Z @ Phi_2 @ Z.T @ X) @ X.T @ Z @ Phi_2 @ Z.T @ ln_wage

u = y - X @ beta_gmm
sigma_matrix = np.diag(u ** 2)  # 构建对角权重矩阵
# sigma_squared = u.T @ u / (len(y) - X.shape[1])

temp1 = np.linalg.inv(X.T @ Z @ Phi_2 @ Z.T @ X) @ X.T @ Z @ Phi_2 @ Z.T
beta_var = temp1 @ sigma_matrix @ temp1.T
beta_var_diag = np.diag(beta_var)

# 计算标准误差，并输出
beta_gmm_std = np.sqrt(beta_var_diag)
print(f'cons: {beta_gmm[0]:.4f}  {beta_gmm_std[0]:.4f}')
print(f'educ: {beta_gmm[1]:.4f}  {beta_gmm_std[1]:.4f}')
print(f'exper: {beta_gmm[2]:.4f}  {beta_gmm_std[2]:.4f}')
print(f'expersq: {beta_gmm[3]:.4f}  {beta_gmm_std[3]:.4f}')

cons: 0.0477  0.4277
educ: 0.0611  0.0332
exper: 0.0451  0.0154
expersq: -0.0009  0.0004


In [13]:
from linearmodels.iv import IVGMM

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IVGMM(ln_wage, exog, endog, instruments)
results1 = model.fit(cov_type = "heteroskedastic")
print(results1.summary)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   wage   R-squared:                      0.1354
Estimator:                     IV-GMM   Adj. R-squared:                 0.1293
No. Observations:                 428   F-statistic:                    18.655
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0003
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:       heteroskedastic                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0477     0.4277     0.1114     0.9113     -0.7907      0.8860
exog.1         0.0451     0.0154     2.9269     0.00

GMM estimator（稳健标准误） 计算公式：$\hat{\beta}^{(2)} = (X' Z \Phi^{(2)} Z' X)^{-1} X' Z \Phi^{(2)} Z' y$
   - 标准式（$\Omega$ 为方差）：
$$Var[\beta^{(2)}] = (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}Z' \Omega Z\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

   - 展开式：
$$Var[\beta] = (X'Z\Phi^{(2)}Z'X)^{-1} X'Z\Phi^{(2)}\left( \sum_{i=g}^{G} Z_g^{\prime} \widehat{u}_g \widehat{u}_g^{\prime} Z_g \right)\Phi^{(2)}Z'X(X'Z\Phi^{(2)}Z'X)^{-1}$$

In [14]:
X = np.c_[cons, mroz['educ'], exper, expersq]
Z = np.c_[cons, mroz['exper'], mroz['expersq'], mroz['motheduc'], mroz['fatheduc']]
Phi_1 = np.linalg.inv(Z.T @ Z)
beta_2sls = np.linalg.inv(X.T @ Z @ Phi_1 @ Z.T @ X) @ X.T @ Z @ Phi_1 @ Z.T @ ln_wage

u_1st = ln_wage - X @ beta_2sls
u_1st = np.array(u_1st)
u_1st_reshaped = u_1st[:, np.newaxis]  # 添加一个新的轴，使其成为 (428, 1)
Z_new= Z*u_1st_reshaped
Phi_2 = np.linalg.inv(Z_new.T @ Z_new / len(ln_wage) )
beta_gmm = np.linalg.inv(X.T @ Z @ Phi_2 @ Z.T @ X) @ X.T @ Z @ Phi_2 @ Z.T @ ln_wage

u = y - X @ beta_gmm


# 聚类稳健标准误（残差项不相关，但存在异方差）
N = X.shape[0]  # 总样本数
K = X.shape[1]  # 变量个数

cluster_ids = mroz['city'] # 我们按 'city' 进行聚类
cluster_labels = np.unique(cluster_ids)  # 获取唯一 cluster
G = len(cluster_labels)  # 计算 cluster 数量
S = np.zeros((Z.shape[1], Z.shape[1]))  # 形状为 (K, K) 的零矩阵

# 遍历每个 cluster，计算贡献
for g in cluster_labels:
    idx = (cluster_ids == g)  # 选出属于 cluster g 的观测
    X_g = X[idx, :]  # 选取该 cluster 的 X 矩阵
    Z_g = Z[idx, :]  # 选取该 cluster 的 X 矩阵
    u_g = u[idx].to_numpy().reshape(-1, 1)  # 其中 -1 表示自动计算行数，1 表示只有一列，即转换为列向量。
    S += Z_g.T @ u_g @ u_g.T @ Z_g

temp1 = Phi_2
temp2 = np.linalg.inv(X.T @ Z @ temp1 @ Z.T @ X) @ X.T @ Z  @ temp1

# 计算聚类稳健标准误的方差估计
# dof_correction = (G / (G - 1)) * ((N - 1) / (N - K))  # 附加自由度修正
# beta_var_cluster = temp2 @ S @ temp2.T * dof_correction # 进行自由度修正
beta_var_cluster = temp2 @ S @ temp2.T  # 不进行自由度修正
beta_var_diag = np.diag(beta_var_cluster)  # 取主对角线方差
beta_std = np.sqrt(beta_var_diag)  # 计算标准误差

print(f'cons: {beta[0]:.4f}  {beta_std[0]:.4f}')
print(f'educ: {beta[1]:.4f}  {beta_std[1]:.4f}')
print(f'exper: {beta[2]:.4f}  {beta_std[2]:.4f}')
print(f'expersq: {beta[3]:.4f}  {beta_std[3]:.4f}')

cons: 0.0481  0.1217
educ: 0.0614  0.0131
exper: 0.0442  0.0095
expersq: -0.0009  0.0002


In [15]:
from linearmodels.iv import IVGMM

exog = np.c_[cons,exper,expersq]
endog = mroz['educ']
instruments = np.c_[mroz['motheduc'], mroz['fatheduc']]
model = IVGMM(ln_wage, exog, endog, instruments)
results1 = model.fit(cov_type='clustered', clusters=mroz['city'])
print(results1.summary)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   wage   R-squared:                      0.1354
Estimator:                     IV-GMM   Adj. R-squared:                 0.1293
No. Observations:                 428   F-statistic:                 2.955e+14
Date:                Wed, Mar 19 2025   P-value (F-stat)                0.0000
Time:                        20:16:05   Distribution:                  chi2(3)
Cov. Estimator:             clustered                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog.0         0.0477     0.1217     0.3916     0.6953     -0.1908      0.2861
exog.1         0.0451     0.0095     4.7546     0.00